In [ ]:
import numpy as np
import pandas as pd
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
import pandas as pd
import numpy as np
import lightgbm
from IPython.display import display
from sklearn.model_selection import train_test_split, GroupKFold, KFold
from sklearn.metrics import mean_absolute_error
import optuna
from tensorflow import keras
import tensorflow as tf
from sklearn.preprocessing import normalize

In [ ]:
df = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test_df = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

In [ ]:
def prepare_dataset(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] /df['count']
    df['breath_id_lag']=df['breath_id'].shift(1).fillna(0)
    df['breath_id_lag2']=df['breath_id'].shift(2).fillna(0)
    df['breath_id_lagsame']=np.select([df['breath_id_lag']==df['breath_id']],[1],0)
    df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']],[1],0)
    df['u_in_lag'] = df['u_in'].shift(1).fillna(0)
    df['u_in_lag'] = df['u_in_lag']*df['breath_id_lagsame']
    df['u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['u_in_lag2'] = df['u_in_lag2']*df['breath_id_lag2same']
    df['u_out_lag2'] = df['u_out'].shift(2).fillna(0)
    df['u_out_lag2'] = df['u_out_lag2']*df['breath_id_lag2same']
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['RC'] = df['R']+df['C']
    df = pd.get_dummies(df)
    return df

In [ ]:
train_y = df[['pressure']].to_numpy().reshape(-1, 80)
train_x = prepare_dataset(df[['id','breath_id','R','C','time_step','u_in','u_out']])
test_x = prepare_dataset(test_df)
train_x

In [ ]:
train_x.drop(['id', 'breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same'], axis=1, inplace=True)
test_x.drop(['id', 'breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same'], axis=1,inplace=True)



In [ ]:
train_y.shape

In [ ]:
train_x = np.array(train_x)
test_x = np.array(test_x)

train_x = train_x.reshape(-1, 80, train_x.shape[-1])
test_x = test_x.reshape(-1, 80, test_x.shape[-1])

In [ ]:
test_x.shape

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=2021)
test_preds = []
for fold, (train_idx, test_idx) in enumerate(kf.split(train_x, train_y)):
    print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
    X_train, X_valid = train_x[train_idx], train_x[test_idx]
    y_train, y_valid = train_y[train_idx], train_y[test_idx]
    scheduler = tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 200*((len(X_train)*0.8)/1024), 1e-5)
    model = keras.models.Sequential([
        keras.layers.Input(shape=(80, 26)),
        keras.layers.Bidirectional(keras.layers.LSTM(200, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(150, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True)),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(1),
    ])
    model.compile(optimizer="adam", loss="mae")
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=180, batch_size=256, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler)])
    model.save(f'Fold{fold+1} RNN Weights')
    test_preds.append(model.predict(test_x).squeeze().reshape(-1, 1).squeeze())

In [ ]:
model.summary()

In [ ]:
test_preds

In [ ]:
submission_df = pd.DataFrame([], columns=['id','pressure' ])
submission_df['id'] =test_df['id']
submission_df['pressure'] = test_preds[0]

In [ ]:
submission_df

In [ ]:
submission_df.to_csv('./submission.csv',index=False)

# Please Upvote if you Like my work.